<a href="https://colab.research.google.com/github/ryderwishart/biblical-machine-learning/blob/main/wip/fine_tune_alpaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip install transformers sentencepiece datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset

# Dataset loading and preprocessing

In [3]:
dataset = load_dataset("tatsu-lab/alpaca")

# Split the dataset into train, test, and eval sets
split_dataset = dataset['train'].train_test_split(test_size=0.15)

# Split the dataset into train, test, and eval sets
split_dataset['test'], split_dataset['eval'] = split_dataset['test'].train_test_split(test_size=0.15).values()


# Print the sizes of the datasets
print(f"Train set size: {len(split_dataset['train'])}")
print(f"Test set size: {len(split_dataset['test'])}")
print(f"Eval set size: {len(split_dataset['eval'])}")

  0%|          | 0/1 [00:00<?, ?it/s]

Train set size: 44201
Test set size: 6630
Eval set size: 1171


In [4]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 44201
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 6630
    })
    eval: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 1171
    })
})

# Choose model and tokenizer

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM

tokenizer = LlamaTokenizer.from_pretrained("chainyo/alpaca-lora-7b")
model = LlamaForCausalLM.from_pretrained("chainyo/alpaca-lora-7b")


In [1]:
def preprocess_data(df):
    tokenized_inputs = []
    tokenized_outputs = []
    
    for _, row in df.iterrows():
        instruction = row["instruction"]
        input_text = row["input"]
        output_text = row["output"]
        
        if input_text:
            # NOTE: this is where we can define how we want to frame the prompt
            input_string = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_text}

### Response:"""
        else:
            input_string = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:""" 
        tokenized_input = tokenizer.encode(input_string, return_tensors="pt")
        tokenized_output = tokenizer.encode(output_text, return_tensors="pt")

        tokenized_inputs.append(tokenized_input)
        tokenized_outputs.append(tokenized_output)
        
    return tokenized_inputs, tokenized_outputs

train_inputs, train_outputs = preprocess_data(split_dataset['train'])
val_inputs, val_outputs = preprocess_data(split_dataset['test'])


NameError: ignored

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]

train_dataset = CustomDataset(train_inputs, train_outputs)
val_dataset = CustomDataset(val_inputs, val_outputs)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()


# Evaluation

Goal: use an 'eval' split to compare the answer in the dataset with the answer generated by our now fine-tuned model.

Ryder will help come up with possible answer-quality scoring mechanisms, likely along different dimensions.

Initially, we can just do a cosine distance between the embedding of the generated answer and the embedding of the answer we have in the dataset (which, for now, we will just assume is the 'correct' answer).